In [ ]:
# ========================
# 1) Install & Import
# ========================
!pip -q install yfinance statsmodels tensorflow

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

plt.style.use("seaborn-v0_8")

# ========================
# 2) Load Data
# ========================
# Download Bitcoin price (last 2 years)
btc = yf.download("BTC-USD", start="2022-01-01", end="2024-12-31")
btc = btc[["Close"]].dropna()
btc.head()
python
Copy
Edit
# ========================
# 3) ARIMA Forecasting
# ========================

# Fit ARIMA model
model = ARIMA(btc["Close"], order=(5,1,2))  # (p,d,q) tune as needed
fit = model.fit()

# Forecast next 30 days
forecast = fit.forecast(steps=30)

plt.figure(figsize=(12,6))
plt.plot(btc["Close"], label="Historical Price")
plt.plot(pd.date_range(btc.index[-1], periods=30, freq="D"), forecast, label="ARIMA Forecast")
plt.legend()
plt.title("BTC Price Forecast (ARIMA)")
plt.show()
python
Copy
Edit
# ========================
# 4) LSTM Forecasting
# ========================

# Scale values to 0-1
scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(btc["Close"].values.reshape(-1,1))

# Create sequences
X, y = [], []
window = 60  # lookback window (60 days)
for i in range(window, len(scaled)):
    X.append(scaled[i-window:i, 0])
    y.append(scaled[i, 0])
X, y = np.array(X), np.array(y)

# Reshape for LSTM (samples, timesteps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Build LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1],1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

model.compile(optimizer="adam", loss="mean_squared_error")

# Train with early stopping
es = EarlyStopping(monitor="loss", patience=3, restore_best_weights=True)
history = model.fit(X, y, batch_size=32, epochs=15, verbose=1, callbacks=[es])

# ========================
# 5) Predictions
# ========================

# Predict on training data
train_preds = model.predict(X)
train_preds = scaler.inverse_transform(train_preds)
real_prices = btc["Close"].values[window:]

# Forecast next 30 days
last_seq = scaled[-window:]
predictions = []
cur_seq = last_seq.reshape(1, window, 1)

for _ in range(30):  # forecast 30 days ahead
    next_pred = model.predict(cur_seq)[0,0]
    predictions.append(next_pred)
    cur_seq = np.append(cur_seq[:,1:,:], [[[next_pred]]], axis=1)

future_preds = scaler.inverse_transform(np.array(predictions).reshape(-1,1))

# ========================
# 6) Plot results
# ========================
plt.figure(figsize=(12,6))
plt.plot(btc.index[window:], real_prices, label="Historical Price")
plt.plot(btc.index[window:], train_preds.flatten(), label="LSTM Fitted")
plt.plot(pd.date_range(btc.index[-1], periods=30, freq="D"), future_preds, label="LSTM Forecast")
plt.legend()
plt.title("BTC Price Forecast (LSTM)")
plt.show()